In [1]:
# Import of packages and libraries:
import pandas as pd
import xml.etree.cElementTree as et
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import gensim
from keras.layers import Input
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.layers import Dense, concatenate, Activation, Dropout
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalMaxPooling1D
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

/Users/juan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# XML Parser:

In [2]:
def xml_parser(path):
    d = dict()
    root = et.parse(path)
    rows = root.findall('.//column')
    for row in rows:
        d.setdefault(list(row.attrib.values())[0],[]).append(row.text)
    df = pd.DataFrame.from_dict(d)
    return df

In [3]:
# Parse data:
data_train = pd.DataFrame()
data_test = pd.DataFrame()
path_train = ['tkk_train_2016.xml', 'bank_train_2016.xml']
path_test = ['tkk_test_etalon.xml', 'banks_test_etalon.xml']
for path in path_train:
    df = xml_parser(path)
    data_train = pd.concat([df, data_train], ignore_index=True)
for path in path_test:
    df = xml_parser(path)
    data_test = pd.concat([df, data_test], ignore_index=True)


In [4]:
data_train

,alfabank,bankmoskvy,beeline,date,gazprom,id,komstar,megafon,mts,raiffeisen,rostelecom,rshb,sberbank,skylink,tele2,text,twitid,uralsib,vtb
0,0,NULL,NaN,1406224554,NULL,1,NaN,NaN,NaN,NULL,NaN,NULL,NULL,NaN,NaN,http://t.co/YEVHuvVGA1 Взять кредит тюмень аль...,492367586156630000,NULL,NULL
1,NULL,NULL,NaN,1406224691,NULL,2,NaN,NaN,NaN,NULL,NaN,NULL,NULL,NaN,NaN,Мнение о кредитной карте втб 24 http://t.co/SB...,492368160923070000,NULL,0
2,NULL,NULL,NaN,1406224798,NULL,3,NaN,NaN,NaN,0,NaN,NULL,NULL,NaN,NaN,«Райффайзенбанк»: Снижение ключевой ставки ЦБ ...,492368608346260000,NULL,NULL
3,NULL,NULL,NaN,1406225412,NULL,4,NaN,NaN,NaN,NULL,NaN,NULL,0,NaN,NaN,Современное состояние кредитного поведения в р...,492371181946030000,NULL,NULL
4,NULL,NULL,NaN,1406226283,NULL,5,NaN,NaN,NaN,NULL,NaN,NULL,1,NaN,NaN,@sawik_shuster @YevhenS Главное чтоб банки СБЕ...,492374836564750000,NULL,1
5,NULL,0,NaN,1406226450,NULL,6,NaN,NaN,NaN,NULL,NaN,NULL,NULL,NaN,NaN,http://t.co/Qr6JbSVTxY Оформить краткосрочный ...,492375537080600000,NULL,NULL
6,NULL,NULL,NaN,1406226505,NULL,7,NaN,NaN,NaN,NULL,NaN,NULL,NULL,NaN,NaN,Самый выгодный автокредит в втб 24 http://t.co...,492375766907510000,NULL,1
7,NULL,NULL,NaN,1406226960,NULL,9,NaN,NaN,NaN,NULL,NaN,NULL,0,NaN,NaN,Кредит иногородним в москве сбербанк http://t....,492377674535680000,NULL,NULL
8,NULL,NULL,NaN,1406226982,NULL,10,NaN,NaN,NaN,NULL,NaN,0,NULL,NaN,NaN,Кредитный калькулятор россельхозбанк чита http...,492377768232250000,NULL,NULL
9,NULL,NULL,NaN,1406227218,NULL,11,NaN,NaN,NaN,NULL,NaN,NULL,NULL,NaN,NaN,http://t.co/h6r6GdBe4H Легко можно получить де...,492378757769220000,NULL,1


# Data Preparation:

In [5]:
data_train.drop(['date', 'id', 'twitid'], axis=1, inplace = True, errors='ignore')
data_train.head()

,alfabank,bankmoskvy,beeline,gazprom,komstar,megafon,mts,raiffeisen,rostelecom,rshb,sberbank,skylink,tele2,text,uralsib,vtb
0,0,NULL,NaN,NULL,NaN,NaN,NaN,NULL,NaN,NULL,NULL,NaN,NaN,http://t.co/YEVHuvVGA1 Взять кредит тюмень аль...,NULL,NULL
1,NULL,NULL,NaN,NULL,NaN,NaN,NaN,NULL,NaN,NULL,NULL,NaN,NaN,Мнение о кредитной карте втб 24 http://t.co/SB...,NULL,0
2,NULL,NULL,NaN,NULL,NaN,NaN,NaN,0,NaN,NULL,NULL,NaN,NaN,«Райффайзенбанк»: Снижение ключевой ставки ЦБ ...,NULL,NULL
3,NULL,NULL,NaN,NULL,NaN,NaN,NaN,NULL,NaN,NULL,0,NaN,NaN,Современное состояние кредитного поведения в р...,NULL,NULL
4,NULL,NULL,NaN,NULL,NaN,NaN,NaN,NULL,NaN,NULL,1,NaN,NaN,@sawik_shuster @YevhenS Главное чтоб банки СБЕ...,NULL,1


In [6]:
data_train.fillna(0)

,alfabank,bankmoskvy,beeline,gazprom,komstar,megafon,mts,raiffeisen,rostelecom,rshb,sberbank,skylink,tele2,text,uralsib,vtb
0,0,NULL,0,NULL,0,0,0,NULL,0,NULL,NULL,0,0,http://t.co/YEVHuvVGA1 Взять кредит тюмень аль...,NULL,NULL
1,NULL,NULL,0,NULL,0,0,0,NULL,0,NULL,NULL,0,0,Мнение о кредитной карте втб 24 http://t.co/SB...,NULL,0
2,NULL,NULL,0,NULL,0,0,0,0,0,NULL,NULL,0,0,«Райффайзенбанк»: Снижение ключевой ставки ЦБ ...,NULL,NULL
3,NULL,NULL,0,NULL,0,0,0,NULL,0,NULL,0,0,0,Современное состояние кредитного поведения в р...,NULL,NULL
4,NULL,NULL,0,NULL,0,0,0,NULL,0,NULL,1,0,0,@sawik_shuster @YevhenS Главное чтоб банки СБЕ...,NULL,1
5,NULL,0,0,NULL,0,0,0,NULL,0,NULL,NULL,0,0,http://t.co/Qr6JbSVTxY Оформить краткосрочный ...,NULL,NULL
6,NULL,NULL,0,NULL,0,0,0,NULL,0,NULL,NULL,0,0,Самый выгодный автокредит в втб 24 http://t.co...,NULL,1
7,NULL,NULL,0,NULL,0,0,0,NULL,0,NULL,0,0,0,Кредит иногородним в москве сбербанк http://t....,NULL,NULL
8,NULL,NULL,0,NULL,0,0,0,NULL,0,0,NULL,0,0,Кредитный калькулятор россельхозбанк чита http...,NULL,NULL
9,NULL,NULL,0,NULL,0,0,0,NULL,0,NULL,NULL,0,0,http://t.co/h6r6GdBe4H Легко можно получить де...,NULL,1


# Get Labels:

In [7]:
def label(row):
    if (row == '1').any():
        label = 1
    elif (row == '0').any():
            label = 0
    else:
        label = -1
    return(label)

In [8]:
data_train['label'] = data_train.apply(label, axis=1)
data_train.label.value_counts()

 0    11832
-1     4145
 1     2058
Name: label, dtype: int64

In [9]:
data_test['label'] = data_test.apply(label, axis=1)
data_test.label.value_counts()

 0    3251
-1    1769
 1     540
Name: label, dtype: int64

# Clean Text:

In [10]:
def clean_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

In [11]:
data_train['clean_text'] = data_train.text.apply(clean_text)
data_test['clean_text'] = data_test.text.apply(clean_text)

In [12]:
data_train

,alfabank,bankmoskvy,beeline,gazprom,komstar,megafon,mts,raiffeisen,rostelecom,rshb,sberbank,skylink,tele2,text,uralsib,vtb,label,clean_text
0,0,NULL,NaN,NULL,NaN,NaN,NaN,NULL,NaN,NULL,NULL,NaN,NaN,http://t.co/YEVHuvVGA1 Взять кредит тюмень аль...,NULL,NULL,0,URL взять кредит тюмень альфа банк
1,NULL,NULL,NaN,NULL,NaN,NaN,NaN,NULL,NaN,NULL,NULL,NaN,NaN,Мнение о кредитной карте втб 24 http://t.co/SB...,NULL,0,0,мнение о кредитной карте втб 24 URL
2,NULL,NULL,NaN,NULL,NaN,NaN,NaN,0,NaN,NULL,NULL,NaN,NaN,«Райффайзенбанк»: Снижение ключевой ставки ЦБ ...,NULL,NULL,0,райффайзенбанк снижение ключевой ставки цб на ...
3,NULL,NULL,NaN,NULL,NaN,NaN,NaN,NULL,NaN,NULL,0,NaN,NaN,Современное состояние кредитного поведения в р...,NULL,NULL,0,современное состояние кредитного поведения в р...
4,NULL,NULL,NaN,NULL,NaN,NaN,NaN,NULL,NaN,NULL,1,NaN,NaN,@sawik_shuster @YevhenS Главное чтоб банки СБЕ...,NULL,1,1,USER USER главное чтоб банки сбер и втб
5,NULL,0,NaN,NULL,NaN,NaN,NaN,NULL,NaN,NULL,NULL,NaN,NaN,http://t.co/Qr6JbSVTxY Оформить краткосрочный ...,NULL,NULL,0,URL оформить краткосрочный кредит оао банк москвы
6,NULL,NULL,NaN,NULL,NaN,NaN,NaN,NULL,NaN,NULL,NULL,NaN,NaN,Самый выгодный автокредит в втб 24 http://t.co...,NULL,1,1,самый выгодный автокредит в втб 24 URL
7,NULL,NULL,NaN,NULL,NaN,NaN,NaN,NULL,NaN,NULL,0,NaN,NaN,Кредит иногородним в москве сбербанк http://t....,NULL,NULL,0,кредит иногородним в москве сбербанк URL
8,NULL,NULL,NaN,NULL,NaN,NaN,NaN,NULL,NaN,0,NULL,NaN,NaN,Кредитный калькулятор россельхозбанк чита http...,NULL,NULL,0,кредитный калькулятор россельхозбанк чита URL
9,NULL,NULL,NaN,NULL,NaN,NaN,NaN,NULL,NaN,NULL,NULL,NaN,NaN,http://t.co/h6r6GdBe4H Легко можно получить де...,NULL,1,1,URL легко можно получить денежный кредит ы втб...


# TF-IDF Vectorizer: 

In [13]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(data_train.text)
X_train.shape

/Users/juan/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(18035, 35957)

In [14]:
X_test = vectorizer.transform(data_test.text)
X_test.shape

/Users/juan/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(5560, 35957)

# Logistic Regression Model:

In [15]:
model = LogisticRegression(multi_class='ovr')

In [16]:
model.fit(X_train, data_train.label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
score = model.score(X_test, data_test.label)
print('Model Accuracy:', (score*100), '%')

Model Accuracy: 70.70143884892086 %


# Word2Vec:

In [18]:
# Merge train & test data:
data = pd.concat([data_train, data_test], ignore_index=True)

# Split text:
split_text = data.clean_text.apply(lambda sent: sent.lower().split())

## Training W2V on tweet texts:

In [19]:
w2v = gensim.models.Word2Vec(sentences=split_text, sg=1, min_count=1)

In [20]:
# Check # of unique words:
keys = len(list(w2v.wv.vocab.keys()))
keys

28652

In [21]:
split_text.apply(lambda row: len(row)).value_counts()

6     2386
7     1937
8     1876
5     1848
9     1681
10    1463
11    1411
12    1085
13    1040
16     947
17     927
14     911
15     903
18     880
19     783
20     713
4      696
21     601
22     426
23     312
3      212
24     203
25     116
2       95
26      75
27      33
28      16
29       8
1        5
30       4
32       2
Name: clean_text, dtype: int64

## Transforming Data:

In [22]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.clean_text)
X_train = tokenizer.texts_to_sequences(data_train.clean_text)
X_test = tokenizer.texts_to_sequences(data_test.clean_text)

In [23]:
X_train = pad_sequences(X_train, maxlen=30)
X_test = pad_sequences(X_test, maxlen=30)

In [24]:
X_train

array([[    0,     0,     0, ...,   924,    31,    14],
       [    0,     0,     0, ...,    13,    26,     1],
       [    0,     0,     0, ...,  1996,   208, 13012],
       ...,
       [    0,     0,     0, ...,    20,   601,   532],
       [    0,     0,     0, ...,    20,   601,   532],
       [    0,     0,     0, ...,    20,   601,   532]], dtype=int32)

## Embedding Matrix:

In [25]:
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
vec = []
for word, i in tokenizer.word_index.items():
    if word in w2v.wv.vocab:
        embedding_vector = w2v.wv.get_vector(word)
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        vec.append(word)

In [26]:
embedding_matrix.shape

(28653, 100)

## CNN Model:

In [27]:
n = 30
tweet_input = Input(shape=(n,), dtype='int32')
inp = Embedding(keys+1, 100, input_length=n,
                weights=[embedding_matrix], trainable=False)(tweet_input)

In [28]:
#del branches

In [29]:
branches = []

for size, filters_count in [(2, 10), (3, 10), (4, 10), (5, 10)]:
    for i in range(filters_count):
        # Add Conv. layer
        branch = Conv1D(filters=1, kernel_size=size, padding='valid', activation='relu')(inp)
        # Subsampling layer
        branch = GlobalMaxPooling1D()(branch)
        branches.append(branch)
x = concatenate(branches, axis=1) 
drop_1 = Dropout(0.2)(x)

# Now flatten to 1D, apply FC > Softmax (with dropout) > softmax
hidden = Dense(30, activation='softmax')(drop_1)
drop_3 = Dropout(0.2)(hidden)
out = Dense(3, activation='softmax')(drop_3)

# Initiate Model
model = Model(input=tweet_input, output=out) 

# Compile Model
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy']) 
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 100)      2865300     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 29, 1)        201         embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 29, 1)        201         embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_3 (

/Users/juan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [30]:
# Fit Model
model.fit([X_train], y=to_categorical(data_train.label.as_matrix()+1), verbose=1, epochs=10, batch_size=32)

Epoch 1/10
18035/18035 [==============================] - 28s 2ms/step - loss: 0.8158 - acc: 0.6578
Epoch 2/10
18035/18035 [==============================] - 24s 1ms/step - loss: 0.7192 - acc: 0.7112
Epoch 3/10
18035/18035 [==============================] - 25s 1ms/step - loss: 0.6817 - acc: 0.7202
Epoch 4/10
18035/18035 [==============================] - 24s 1ms/step - loss: 0.6609 - acc: 0.7248
Epoch 5/10
18035/18035 [==============================] - 25s 1ms/step - loss: 0.6418 - acc: 0.7264
Epoch 6/10
18035/18035 [==============================] - 24s 1ms/step - loss: 0.6263 - acc: 0.7323
Epoch 7/10
18035/18035 [==============================] - 25s 1ms/step - loss: 0.6137 - acc: 0.7377
Epoch 8/10
18035/18035 [==============================] - 24s 1ms/step - loss: 0.6024 - acc: 0.7439
Epoch 9/10
18035/18035 [==============================] - 25s 1ms/step - loss: 0.5974 - acc: 0.7438
Epoch 10/10
18035/18035 [==============================] - 24s 1ms/step - loss: 0.5881 - acc: 0.7510

### Results:

In [31]:
model.evaluate(X_test, to_categorical(data_test.label.as_matrix()+1))

5560/5560 [==============================] - 2s 445us/step


[0.7394580670183512, 0.6841726618705036]

In [35]:
data_train.label.as_matrix()+1
y_ = model.predict(X_test)
y_[-1] = [0,0,1]

In [38]:
f_macro = f1_score(to_categorical([np.argmax(x) for x in y_]), to_categorical(data_test.label.as_matrix()+1), average='macro')
f_micro = f1_score(to_categorical([np.argmax(x) for x in y_]), to_categorical(data_test.label.as_matrix()+1), average='micro')
print('F1 Score (Macro):', f_macro)
print('F1 Score (Micro):', f_micro)

F1 Score (Macro): 0.47042022436657494
F1 Score (Micro): 0.6843525179856115
